# GOURMET GUIDE

### Foundation Project 1: Food Recommendation Model

#### Group 6

* Charanjeet Singh - 12220064
* Pooja Nilesh Doshi - 12220028
* Snigdha Debashis Bhattacharjee - 12220067
* Vinayak Dave - 12220047

In [1]:
#!pip install threadpoolctl==3.1.0
#!pip install yellowbrick
#!pip install pca
#!pip install spacy==3.5.2
#!python -m spacy download en

In [2]:
import json
import string
import spacy
import matplotlib as m
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np
import os
import re
import pandas as pd
from pandas import DataFrame
from scipy import stats
from scipy.stats import norm
import seaborn as sns
import threadpoolctl
import warnings
warnings.filterwarnings("ignore")
from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [3]:
train_df = pd.read_excel(r"C:\Users\Green Lantern\OneDrive\Documents\ISB AMPBA Winter 2023\Term 2\Foundational Project 1\FP1_Project 4_Group6_Final\restaurant dataset\zomato_train_df.xlsx")

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9458 entries, 0 to 9457
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    9458 non-null   int64  
 1   rest_id       9458 non-null   int64  
 2   url           9458 non-null   object 
 3   name          9458 non-null   object 
 4   online_order  9458 non-null   bool   
 5   book_table    9458 non-null   bool   
 6   rate          9458 non-null   float64
 7   votes         9458 non-null   int64  
 8   location      9458 non-null   object 
 9   rest_type     9458 non-null   object 
 10  dish_liked    9458 non-null   object 
 11  cuisines      9458 non-null   object 
 12  cost          9458 non-null   int64  
 13  reviews_list  9399 non-null   object 
 14  cost_cluster  9458 non-null   int64  
dtypes: bool(2), float64(1), int64(5), object(7)
memory usage: 979.2+ KB


In [5]:
#Deleting Unnnecessary Columns
train_df = train_df.drop(['Unnamed: 0'],axis=1)

In [6]:
train_df.head()

,rest_id,url,name,online_order,book_table,rate,votes,location,rest_type,dish_liked,cuisines,cost,reviews_list,cost_cluster
0,48586,https://www.zomato.com/bangalore/mudpipe-cafe-...,Mudpipe Cafe,True,False,4.2,745,Cunningham Road,Cafe,"Sandwiches, Lemonade, Nachos, Mozzarella Stick...",Italian,750,visited place afternoon preferred place serves...,1
1,19653,https://www.zomato.com/bangalore/tarbouche-kor...,Tarbouche,True,False,4.1,268,Koramangala 5th Block,Kiosk,"Falafel, Lamb Kebab, Rolls, Fattoush, Teriyaki...",Turkish,300,stall located koramangala 5th block place serv...,2
2,4981,https://www.zomato.com/bangalore/olive-bar-and...,Olive Bar And Kitchen,False,True,4.5,2194,Richmond Road,Casual Dining,"Salads, Cocktails, Hazelnut Cake, Pepperoni Pi...",European,100,firstly change name place olive fine dining re...,0
3,36600,https://www.zomato.com/bangalore/axomi-taste-o...,Axomi Taste Of Assam,True,False,4.1,616,Koramangala 7th Block,Casual Dining,"Thali, Duck Curry, Fish, Brinjal Fry, Aloo Pit...",Assamese,500,it ...,2
4,46081,https://www.zomato.com/bangalore/pecos-stones-...,Pecos Stones,False,True,4.2,1096,Indiranagar,Pub,"Chicken Curry, Chicken Dosa, Sea Food, Masala ...",Finger Food,100,one oldest pubs indiranagarnvery chilled ambia...,0


#### MODEL ON TRAINING DATASET

In [7]:
train_df.set_index('name', inplace=True)

In [8]:
indices = pd.Series(train_df.index)

In [9]:
 cost_clusters = {
    0: [100.0, 250.0],
    1: [300.0, 550.0],
    2: [600.0, 950.0]}
train_df['cost_cluster']=''
train_df.loc[(train_df['cost'] >= 100.0) & (train_df['cost'] <= 250.0), 'cost_cluster'] = 0
train_df.loc[(train_df['cost'] >= 300.0) & (train_df['cost'] <= 550.0), 'cost_cluster'] = 1
train_df.loc[(train_df['cost'] >= 600.0) & (train_df['cost'] <= 950.0), 'cost_cluster'] = 2

In [10]:
def user_recommendation():
    unique_loc = train_df['location'].unique()
    print("Choose Location:")
    for i, loc in enumerate(unique_loc, start=1):
        print(f"{i}. {loc}")
        
    location = input("Where are you located? ")
    
    unique_cuisines_df = train_df[train_df['location'] == location]['cuisines']
    unique_cuisines = unique_cuisines_df.unique()
    if unique_cuisines_df.empty:
        print("Choose your Location")
        return
    
    print("Choose Cuisine:")
    for i, loc in enumerate(unique_cuisines, start=1):
        print(f"{i}. {loc}")
        
    cuisine = input("What are you craving for? ")
    
    Rest_Name_df = train_df[(train_df['location'] == location) & (train_df['cuisines'] == cuisine)]
    if Rest_Name_df.empty:
        print("Choose your preferred Cuisine")
        return
    Rest_Name = Rest_Name_df['rest_type'].unique()
    print("Choose Restaurant Type:")
    for i, loc in enumerate(Rest_Name, start=1):
        print(f"{i}. {loc}")
    
    rest_type = input("What is your choice of restaurant? ")
    
    rest_type = rest_type.lstrip()
    rest_type = rest_type.rstrip()
    location= location.lstrip()
    location= location.rstrip()
    cuisine=cuisine.lstrip()
    cuisine=cuisine.rstrip()
    

    #Filtering the dataframe based on user preferences
    filtered_df = train_df[(train_df['location'] == location) & (train_df['cuisines'] == cuisine) & (train_df['rest_type'] == rest_type)]
         
    if filtered_df.empty:
                    print("4")
                    print("No Restaurants Found")
                    return
    
    cost_clusters = {
    0: [100.0, 250.0],
    1: [300.0, 550.0],
    2: [600.0, 950.0]}
    
    
    cost_cluster = []
    ranges = []
    cost_cluster = filtered_df['cost_cluster'].unique()
    # Prompt user to select a cost range
    
    for i in cost_cluster:
        for cluster, cost_range in cost_clusters.items():
            if cluster == i:
                #print(f"{cluster}. Rs.{cost_range[0]} to Rs.{cost_range[1]}")
                ranges.append(f"{cluster}. Rs.{cost_range[0]} to Rs.{cost_range[1]}")
    print("Cost for Two:")
    for i in reversed(ranges):
        print(i)
            
    
    cost_range = int(input("Cost for Two: "))
    
    # Filter the data based on selected cost-range
    cost_filter = (filtered_df['cost'] >= cost_clusters[cost_range][0]) & (filtered_df['cost'] <= cost_clusters[cost_range][1])
    filtered_df = filtered_df[cost_filter]
    
    if filtered_df.empty:
        print("No Restaurants Found")
        return
    
    
    num_rows = filtered_df.shape[0]        
    if num_rows == 1:
        
        name =filtered_df.index.tolist()
        print(name)
        recommendations_df = pd.DataFrame({'Restaurant': name, 'Rating': filtered_df['rate']})
        recommendations_df = recommendations_df.reset_index(drop=True)
    
    else:        
    #Creating tf-idf matrix for reviews
        tfidf_reviews = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
        tfidf_reviews_matrix = tfidf_reviews.fit_transform(filtered_df['reviews_list'])
    
    #Computing cosine similarity for reviews
        cosine_reviews = cosine_similarity(tfidf_reviews_matrix, tfidf_reviews_matrix)
    
    #Finding the index of the restaurant entered
        idx = filtered_df.index[0]
        unique_indexes = filtered_df.index.unique()
    
    # Find the restaurants with a similar cosine-sim value for reviews and order them from biggest number
        if idx in unique_indexes :
            idx = unique_indexes.get_loc(idx)
            sim_scores_reviews = list(enumerate(cosine_reviews[idx]))
            sim_scores_reviews = sorted(list(sim_scores_reviews), key=lambda x: x[1], reverse=True)
        #Extracting top 5 restaurant indexes with a similar cosine-sim value
            top5_indexes_reviews = [i[0] for i in sim_scores_reviews[1:6]]
        
        #Names of the top 5 restaurants based on reviews
            recommend_restaurants_reviews = list(filtered_df.index[top5_indexes_reviews].unique())
        
        # Ratings of the top 5 restaurants based on reviews
            recommend_ratings_reviews = [np.unique(filtered_df.loc[restaurant]['rate']) for restaurant in recommend_restaurants_reviews]

        
        if not recommend_restaurants_reviews:
            print("5")
            print("No Restaurants Found")
            return
    
    #Creating a DataFrame to store the recommendations and ratings
        recommendations_df = pd.DataFrame({'Restaurant': recommend_restaurants_reviews, 'Rating': recommend_ratings_reviews})
    
    print("Restaurants you may also like: ")
    print(recommendations_df)
    
    #User selects a restaurant
    selected_restaurant = input("Select a restaurant from the list above: ")
    
    if selected_restaurant not in recommendations_df['Restaurant'].values:
                print("Invalid restaurant selection")
                return
    
    #Getting the index of the selected restaurant
    selected_index = recommendations_df[recommendations_df['Restaurant'] == selected_restaurant].index[0]
    #Getting the recommended dishes for the selected restaurant
    selected_recommend_dishes = filtered_df.loc[recommendations_df.loc[selected_index, 'Restaurant'], 'dish_liked']
    if isinstance(selected_recommend_dishes, str):
         selected_recommend_dishes = [selected_recommend_dishes]
    else:
    # Remove duplicate values from the Series
        selected_recommend_dishes = selected_recommend_dishes.drop_duplicates().tolist()

# Concatenate the dishes from all rows into a single string
    all_dishes = ','.join(selected_recommend_dishes)

# Split the concatenated string by comma and create a list of individual dishes
    dishes_list = [dish.strip() for dish in all_dishes.split(',')]

# Get unique dishes by converting the list to a set and back to a list
    unique_dishes = list(set(dishes_list))

# Create a DataFrame with each unique dish in a separate row
    df = pd.DataFrame({'Dishes': unique_dishes})
    print("Dishes you will like in", selected_restaurant, ":")
    for dish in df['Dishes']:
        print(dish)
    
user_recommendation()

Choose Location:
1. Cunningham Road
2. Koramangala 5th Block
3. Richmond Road
4. Koramangala 7th Block
5. Indiranagar
6. BTM
7. JP Nagar
8. Sarjapur Road
9. Jayanagar
10. Ulsoor
11. Residency Road
12. Church Street
13. Banaswadi
14. Central Bangalore
15. Brigade Road
16. Brookefield
17. Domlur
18. Whitefield
19. Banashankari
20. Electronic City
21. HSR
22. Kumaraswamy Layout
23. MG Road
24. Kammanahalli
25. St. Marks Road
26. Koramangala 6th Block
27. Lavelle Road
28. Koramangala 1st Block
29. Kalyan Nagar
30. Infantry Road
31. Marathahalli
32. Sanjay Nagar
33. Koramangala 4th Block
34. Rajajinagar
35. Bommanahalli
36. Jeevan Bhima Nagar
37. Bannerghatta Road
38. Sankey Road
39. New BEL Road
40. Bellandur
41. Basavanagudi
42. Frazer Town
43. Vasanth Nagar
44. Shanti Nagar
45. Thippasandra
46. Basaveshwara Nagar
47. Langford Town
48. Malleshwaram
49. Old Airport Road
50. South Bangalore
51. Ejipura
52. Commercial Street
53. ITPL Main Road, Whitefield
54. HBR Layout
55. Nagawara
56. Kora